In [1]:
#!pip install psycopg2-binary
#!pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd
import random
import psycopg2
import datetime
import numpy as np
import json
from typing import Optional, List
from dataclasses import dataclass, asdict

In [3]:
settings = json.loads(open(".credentials.json").read())

In [4]:
settings = json.loads(open(".credentials.json").read())
schema = "cds_cdm"
con = psycopg2.connect(**settings, options=f"-c search_path={schema}")

In [5]:
cursor = con.cursor() 

In [6]:
# get maximal patient_id from DB

cursor.execute("SELECT MAX(person_id) FROM person");
startPatientID = cursor.fetchone()[0]
if startPatientID is None:
    startPatientID = 0
else:
    startPatientID += 1
print(startPatientID)

66


In [7]:
# function to create a list (of patients)

def createlist(num_pat):
    return list(range(startPatientID, startPatientID + num_pat + 1))

In [8]:
###############################
####### SET VARIABLES #########
###############################

# set number of patients to five
patient_list = createlist(5)

# choose patient for one, limited example 
patient_id = patient_list[0]

# entries to choose from

# regarding person
gender_list = [8532, 8507, 8521] # Female, Male, Other
birthyear_range = range(1920,2003)
race_list = [8515, 38003598, 38003614] # Asian, Black, European

#regarding visits, creating a list of all dates from 2020 and 2021
date1 = datetime.date(2020, 1, 1)
date2 = datetime.date(2021, 12, 31)
res_dates = [date1]
while date1 != date2:
    date1 += datetime.timedelta(days=1)
    res_dates.append(date1)
    
# regarding diagnoses and treatment
intensive_bin = [32037] # "Intensive Care"
beatm_bin = [4230167] # "Artificial respiration"
prone_bin = [4196006] # "Placing subject in prone position"
condition_list = [37311061, 444247, 4009307, 432870, 440417, 4195694] # "COVID-19"; "Venous Thrombosis"; "Heparin-induced thrombocytopenia with thrombosis"; "Thrombocytopenic disorder"; "Pulmonary Embolism"; "Acute respiratory distress syndrome" 
obs_list = [4169185, 4170358] # "Allergy to heparin"; "Allergy to heparinoid"
lab_list = [3048530, 3013466, 3029943] # "Fibrin D-dimer DDU [Mass/volume] in Platelet poor plasma"; "aPTT in Blood by Coagulation assay"; "Horowitz index in Arterial blood"; 
beatm_list = [3020716, 3017878, 21490650, 21490855] # "Inhaled oxygen concentration"; "Tidal volume.spontaneous+mechanical/Body weight [Volume/mass] --on ventilator"; "Pressure max Respiratory system airway --during inspiration"; "PEEP Respiratory system --on ventilator"
drug_list = [1367571, 1322207, 1301065, 1301025, 19001014, 19016072, 1315865] # "heparin"; "argatroban"; "dalteparin"; "enoxaparin"; "nadroparin"; "certoparin"; "fondaparinux"; 


In [9]:
patient_list

[66, 67, 68, 69, 70, 71]

In [10]:
@dataclass
class Person:
    person_id: int
    gender_concept_id: Optional[int] = None
    year_of_birth : Optional[int] = None
    month_of_birth : Optional[int] = None
    day_of_birth: Optional[int] = None
    race_concept_id: Optional[int] = None
    ethnicity_concept_id: Optional[int] = None
    
    def __post_init__(self):
        self.gender_concept_id = random.choice(gender_list) #if self.gender_concept_id is None else self.gender_concept_id
        self.year_of_birth = random.choice(birthyear_range)
        self.month_of_birth = random.choice(range(1,12))
        
        if self.month_of_birth in [1, 3, 5, 7, 8, 10, 12]:
            self.day_of_birth = random.choice(range(1,31))
        elif self.month_of_birth in [4, 6, 9, 11]:
            self.day_of_birth = random.choice(range(1,30))
        elif self.month_of_birth == 2:
            self.day_of_birth = random.choice(range(1,28))
        
        self.race_concept_id = random.choice(race_list)
        self.ethnicity_concept_id = 0 # always set to zero!


In [11]:
# creating only entirely intensive-care and non-intensive care visits at the moment

@dataclass
class Visit_occurrence:
    person_id: int
    visit_concept_id: Optional[int] = None
    visit_start_date: Optional[datetime.date] = None
    visit_end_date: Optional[datetime.date] = None
    visit_type_concept_id: Optional[int] = None
        
    def __post_init__(self):
        self.visit_concept_id = random.choice([9201, 32037]) # "Inpatient visit" or "Intensive care"
        self.visit_start_date = random.choice(res_dates)
        self.visit_end_date = self.visit_start_date + datetime.timedelta(days = random.choice(range(3,60)))
        self.visit_type_concept_id = 32817

In [12]:
@dataclass
class Procedure_occurrence:
    person_id: int
    procedure_concept_id: int
    procedure_type_concept_id: int
    procedure_date: datetime.date
    procedure_datetime: datetime.datetime
  #  procedure_end_datetime: datetime.datetime


In [13]:
@dataclass
class Drug_exposure:
    person_id: int
    drug_concept_id: int
    drug_exposure_start_date: datetime.date
    drug_exposure_start_datetime: datetime.datetime
    drug_exposure_end_date: datetime.date
    drug_exposure_end_datetime: datetime.datetime
    quantity: int
    drug_type_concept_id: Optional[int] = None
    
    def __post_init__(self):
       self.drug_type_concept_id = 32817
    

In [14]:
@dataclass
class Measurement:
    person_id: int
    measurement_concept_id: int
    measurement_date: datetime.date
    value_as_number: float
    unit_concept_id: int
    measurement_type_concept_id: Optional[int] = None
    
    def __post_init__(self):
       self.measurement_type_concept_id = 32817   


In [15]:
# create drug exposures from drug list. i = number of boluses in case of LWMH/Fondaparinux or i = number of doserate changes in case of continuous infusions
def create_drug_exp2(person_id, v, i) -> List[Drug_exposure]:
    list_of_drugs = []
    begin_drug = v.visit_start_date + datetime.timedelta(days=random.choice(range(4)))
    end_drug = begin_drug + (random.random() * (v.visit_end_date - begin_drug))
#    end_drug = begin_drug + datetime.timedelta(days = random.choice(range(2,30)))
    # create dttm from date, add random number of hours to startdate midnight
    drug_exposure_start_datetime = datetime.datetime.combine(begin_drug, datetime.datetime.min.time()) + datetime.timedelta(hours = random.choice(range(12)))    
    drug_exposure_id = 0
    drug_concept_id = random.choice(drug_list)
    if drug_concept_id in [1367571, 1301065, 19001014, 19016072]:
        drug_unit = 8510
    elif drug_concept_id in [1322207, 1301025, 1315865]:
        drug_unit = 8576

    ingredient_concept_id = drug_concept_id
    valid_start_date = datetime.date(1970, 1, 1)
    # continuous infusion of drugs, quantity referring to [dose] / h 
    if drug_concept_id in [1367571, 1322207]:
        drug_exposure_end_datetime = drug_exposure_start_datetime
        for x in range(i):
            drug_exposure_id += 1 
            drug_exposure_start_datetime = drug_exposure_end_datetime + datetime.timedelta(hours=random.choice(range(2)))
            drug_exposure_start_date = drug_exposure_start_datetime.date() # create date from dttm
            if drug_concept_id == 1367571: # heparin in IE/h
                quantity_h = random.choice(range(200, 900, 100))
            elif drug_concept_id == 1322207: # argatroban in mg/h
                quantity_h = random.choice(range(5, 10, 1))
            drug_exposure_end_datetime = drug_exposure_start_datetime + datetime.timedelta(hours=random.choice(range(1, 12)))
            drug_exposure_end_date = drug_exposure_end_datetime.date()
            duration = drug_exposure_end_datetime - drug_exposure_start_datetime
            duration_h = duration.total_seconds()/3600
            quantity = duration_h * quantity_h
            if drug_exposure_start_date > end_drug:
                break
            list_of_drugs.append(Drug_exposure(person_id, drug_concept_id, drug_exposure_start_date,
                                      drug_exposure_start_datetime, drug_exposure_end_date,
                                      drug_exposure_end_datetime, quantity))
        
    if drug_concept_id in [1301065, 19001014, 19016072, 1301025, 1315865]:
        for x in range(i):
            drug_exposure_id += 1 
            drug_exposure_start_datetime += datetime.timedelta(hours=random.choice(range(4,20)))
            drug_exposure_start_date = drug_exposure_start_datetime.date() # create date from dttm
            if drug_concept_id in [1301065, 19001014]: # dalteparin, nadroparin
                quantity = random.choice(range(3000, 15000, 1000))
            elif drug_concept_id == 19016072: # certoparin
                quantity = random.choice(range(1000, 4000, 500))
            elif drug_concept_id == 1301025: # enoxaparin
                quantity = random.choice(range(10,120,10))
            elif drug_concept_id == 1315865: # fondaparinux
                quantity = random.choice(range(1,4,1))
            else:
                pass
            drug_exposure_end_date = drug_exposure_start_date
            if drug_exposure_start_date > end_drug:
                break

            list_of_drugs.append(Drug_exposure(person_id, drug_concept_id, drug_exposure_start_date,
                          drug_exposure_start_datetime, drug_exposure_end_date,
                          None, quantity))
    
    return list_of_drugs

In [16]:
# create (only one) episode of ventilation or O2 therapy for ICU patients with related parameters. Patients not treated on ICU do not have such parameters
# create procedures here
def create_vent_params_procedure(person_id, v) -> Procedure_occurrence:
    person_id = person_id
    measurement_id = 0
    procedure_occurence_id = 0 # used for ventilation OR oxygen therapy at the moment
    # set different frequencies (x per day)
    freq1 = 24 # used for high-frequency generation of ventilated patients
    freq2 = 2 # used for non-ventilated patients
    freq3 = 4 # unused
    if v.visit_concept_id == 32037:
        print("#########################")
        print("patient is treated on ICU")
        ventilated = random.choice(range(2)) # is there an episode of artificial respiration?
        if ventilated == 1: # create parameters for ventilated patients
            print("patient has episode of invasive VENTILATION")
            print("#########################")
            procedure_concept_id = 4230167
            procedure_type_concept_id = 32817
            begin_vent = v.visit_start_date + datetime.timedelta(days=random.choice(range(3)))
            end_vent = begin_vent + (random.random() * (v.visit_end_date - begin_vent))
            procedure_date = begin_vent
            procedure_datetime = datetime.datetime.combine(procedure_date, datetime.datetime.min.time())
            procedure_end_date = end_vent
        #    procedure_end_datetime = datetime.datetime.combine(procedure_end_date, datetime.datetime.min.time())
           
            return Procedure_occurrence(person_id, procedure_concept_id,
                                      procedure_type_concept_id, procedure_date,
                                      procedure_datetime)
 
        else: # create parameters for non-ventilated patients (on ICU)
            print("patient is treated by oxygen therapy")
            print("#########################")
            procedure_concept_id = 4239130
            procedure_type_concept_id = 32817
            begin_nonvent = v.visit_start_date + datetime.timedelta(days=random.choice(range(3)))
            end_nonvent = begin_nonvent + (random.random() * (v.visit_end_date - begin_nonvent))
            procedure_date = begin_nonvent
            procedure_datetime = datetime.datetime.combine(procedure_date, datetime.datetime.min.time())
            procedure_end_date = end_nonvent
     #       procedure_end_datetime = datetime.datetime.combine(procedure_end_date, datetime.datetime.min.time())
            
            return Procedure_occurrence(person_id, procedure_concept_id,
                          procedure_type_concept_id, procedure_date,
                          procedure_datetime)
        
    else: # patients not on ICU do not have any ventilation parameters (worth discussing whether patients on normal ward can have oxygen therapy)
        return None

In [17]:
# create (only one) episode of ventilation or O2 therapy for ICU patients with related parameters. Patients not treated on ICU do not have such parameters
# create measurements here
def create_vent_params_measurements(person_id, prod, v) -> List[Measurement]:
    list_of_measurements = []
    person_id = person_id
    measurement_id = 0
    # set different frequencies (x per day)
    freq1 = 24 # used for high-frequency generation of ventilated patients
    freq2 = 2 # used for non-ventilated patients
    freq3 = 4 # unused
    if v.visit_concept_id == 32037:
        if prod is not None and prod.procedure_concept_id == 4230167: # create parameters for ventilated patients
            begin_vent = prod.procedure_date
            end_vent = begin_vent + (random.random() * (v.visit_end_date - begin_vent))
            duration_vent = (end_vent - begin_vent).total_seconds()/(60*60*24)            
            for x in range(int(duration_vent)):
                base_datetime = datetime.datetime.combine(begin_vent, datetime.datetime.min.time())
                begin_vent += datetime.timedelta(days=1)
                # create FiO2 values
                measurement_datetime = base_datetime # set measurement_datetime to base
                for x in range(freq1):
                    measurement_id += 1
                    measurement_date = measurement_datetime.date()
                    measurement_datetime += datetime.timedelta(hours = 24/freq1)
                    measurement_concept_id = 3020716 # "Inhaled oxygen concentration"
                    unit_concept_id = 8554
                    value_as_number = random.choice(np.arange(0.2, 1.0, 0.05))           
                    list_of_measurements.append(Measurement(person_id, measurement_concept_id,
                                                           measurement_date, value_as_number,
                                                           unit_concept_id ))
                    
                    
                # create tidal volume per body weight values
                measurement_datetime = base_datetime # set measurement_datetime to base
                for x in range(freq1):
                    measurement_id += 1
                    measurement_date = measurement_datetime.date()
                    measurement_datetime += datetime.timedelta(hours = 24/freq1)
                    measurement_concept_id = 3017878 # "Tidal volume.spontaneous+mechanical/Body weight [Volume/mass] --on ventilator"
                    unit_concept_id = 9571
                    value_as_number = random.choice(range(1,12))
                    list_of_measurements.append(Measurement(person_id, measurement_concept_id,
                                       measurement_date, value_as_number,
                                       unit_concept_id))

                    
                # create maximum inspiratory pressure values
                measurement_datetime = base_datetime # set measurement_datetime to base
                for x in range(freq1):
                    measurement_id += 1
                    measurement_date = measurement_datetime.date()
                    measurement_datetime += datetime.timedelta(hours = 24/freq1)
                    measurement_concept_id = 21490650 # "Pressure max Respiratory system airway --during inspiration"
                    unit_concept_id = 44777590
                    value_as_number = random.choice(range(12,40))
                    list_of_measurements.append(Measurement(person_id, measurement_concept_id,
                           measurement_date, value_as_number,
                           unit_concept_id ))
                    
                    #print("unit_concept_id:", unit_concept_id)
                # create PEEP values
                measurement_datetime = base_datetime # set measurement_datetime to base
                for x in range(freq1):
                    measurement_id += 1
                    measurement_date = measurement_datetime.date()
                    measurement_datetime += datetime.timedelta(hours = 24/freq1)
                    measurement_concept_id = 21490855 # "PEEP Respiratory system --on ventilator"
                    unit_concept_id = 44777590
                    value_as_number = random.choice(range(12,40))
                    list_of_measurements.append(Measurement(person_id, measurement_concept_id,
                                       measurement_date, value_as_number,
                                       unit_concept_id ))

                    
        else: # create parameters for non-ventilated patients (on ICU)
            print("patient is treated by oxygen therapy")
            print("#########################")
            begin_nonvent = prod.procedure_date
            end_nonvent = begin_nonvent + (random.random() * (v.visit_end_date - begin_nonvent))
            duration_nonvent = (end_nonvent - begin_nonvent).total_seconds()/(60*60*24)
            for x in range(int(duration_nonvent)):
                base_datetime = datetime.datetime.combine(begin_nonvent, datetime.datetime.min.time())
                begin_nonvent += datetime.timedelta(days=1)
                # create FiO2 values
                measurement_datetime = base_datetime # set measurement_datetime to base
                for x in range(freq2):
                    measurement_id += 1
                    measurement_date = measurement_datetime.date()
                    measurement_datetime += datetime.timedelta(hours = 24/freq2)
                    measurement_concept_id = 3020716 # "Inhaled oxygen concentration"
                    unit_concept_id = 8554
                    value_as_number = random.choice(np.arange(0.2, 0.5, 0.05))           
                    list_of_measurements.append(Measurement(person_id, measurement_concept_id,
                                       measurement_date, value_as_number,
                                       unit_concept_id))

                    
    else: # patients not on ICU do not have any ventilation parameters (worth discussing whether patients on normal ward can have oxygen therapy)
        pass
    
    return list_of_measurements

In [18]:
# ideally, not done yet: if ARDS or mechanical ventilation is existent, Horowitz Index should be lower, because those patients are sicker
def create_lab_values_measurements(person_id, v) -> List[Measurement]:
    list_of_measurements = []
    # set different frequencies (x per day)
    freq1 = 4 # used for higher-frequency generation of values 
    freq2 = 2 # used for less frequent generation
    freq3 = 1 # once per day
    
    unit_concept_id = 1 # TODO
    
    # set "basetime" to visit_start, i.e. generation of lab values are started up to twelve hours after the start of the visit
    measurement_datetime = datetime.datetime.combine(v.visit_start_date, datetime.datetime.min.time()) + datetime.timedelta(hours=random.choice(range(12)))
    measurement_date = measurement_datetime.date()
    
    visit_duration = v.visit_end_date - v.visit_start_date

    horo_lst = [] # generate a normally distributed list of plausible values for Horowitz Index
    for i in range(500):
        x = np.random.normal(loc = 200, scale = 50)
        horo_lst.append(x)
    ptt_lst = [] # generate a normally distributed list of plausible values for pTT values
    for i in range(500):
        x = np.random.normal(loc = 50, scale = 10)
        ptt_lst.append(x)
    ddimer_lst = [] # generate a binomial distributed list of plausible values for d-dimer
    for i in range(500):
        x = np.random.binomial(40, 0.45)
        x = x/10
        ddimer_lst.append(x)
    if v.visit_concept_id == 32037:
        print("#########################")
        print("patient is treated on ICU")
        
        for x in range(int(visit_duration.total_seconds()/(24*60*69))): # generate per day
            for x in range(freq1): # with frequency 1
                measurement_date = measurement_datetime.date()
                measurement_concept_id = 3029943 # Horowitz Quotient/Index
                measurement_type_concept_id = 32817
                # maybe here introduce if condition_concept_id ARDS/mechanical ventilation is given
                value_as_number = random.choice(horo_lst)
                list_of_measurements.append(Measurement(person_id, measurement_concept_id,
                                        measurement_date, round(value_as_number, 0),
                                       unit_concept_id))
                
                measurement_datetime += datetime.timedelta(hours = 24/freq1)
                
            for x in range(freq3): # with frequency 3
                measurement_date = measurement_datetime.date()
                measurement_concept_id = 3013466 # pTT
                measurement_type_concept_id = 32817
                value_as_number = random.choice(ptt_lst)
                list_of_measurements.append(Measurement(person_id, measurement_concept_id,
                   measurement_date, round(value_as_number, 0),
                   unit_concept_id))
                
                measurement_datetime += datetime.timedelta(hours = 24/freq3)
                
            for x in range(freq3): # with frequency 3
                measurement_date = measurement_datetime.date()
                measurement_concept_id = 3048530 # D-Dimer
                measurement_type_concept_id = 32817
                value_as_number = random.choice(ddimer_lst)
                list_of_measurements.append(Measurement(person_id, measurement_concept_id,
                   measurement_date, round(value_as_number, 0),
                   unit_concept_id))
                
                measurement_datetime += datetime.timedelta(hours = 24/freq3)
                
    if v.visit_concept_id == 9201:
        print("#########################")
        print("patient is treated on normal ward")
        for x in range(int(visit_duration.total_seconds()/(24*60*69))): # generate per day
            for x in range(freq2): # with frequency 2
                measurement_date = measurement_datetime.date()
                measurement_concept_id = 3029943 # Horowitz Quotient/Index
                measurement_type_concept_id = 32817
                # maybe here introduce if condition concerning ARDS/mechanical ventilation
                value_as_number = random.choice(horo_lst)
                list_of_measurements.append(Measurement(person_id, measurement_concept_id,
                   measurement_date, round(value_as_number, 0),
                   unit_concept_id))
                
                measurement_datetime += datetime.timedelta(hours = 24/freq2)
                
            for x in range(freq3): # with frequency 3
                measurement_date = measurement_datetime.date()
                measurement_concept_id = 3013466 # pTT
                measurement_type_concept_id = 32817
                value_as_number = random.choice(ptt_lst)
                list_of_measurements.append(Measurement(person_id, measurement_concept_id,
                   measurement_date, round(value_as_number, 0),
                   unit_concept_id))
                
                measurement_datetime += datetime.timedelta(hours = 24/freq3)
                
            for x in range(freq3): # with frequency 3
                measurement_date = measurement_datetime.date()
                measurement_concept_id = 3048530 # D-Dimer
                measurement_type_concept_id = 32817
                value_as_number = random.choice(ddimer_lst)
                list_of_measurements.append(Measurement(person_id, measurement_concept_id,
                   measurement_date, round(value_as_number, 0),
                   unit_concept_id))
                
                measurement_datetime += datetime.timedelta(hours = 24/freq3)
                
    return list_of_measurements

In [19]:
# ideally, not done yet: only patients who have the diagnosis ARDS and/or a Horowitz index of lower than 150 at least once should be placed in prone positioning
def create_pos_procedure(person_id, v, prod, x = 5) -> List[Procedure_occurrence]: # x = 5
    list_of_procedures = []
    r = random.random() # roll the dice if patient is put in prone positioning
    if r >= 0.5:
        print("#########################")
        print("patient is placed in prone positioning at least once")
        date1 = v.visit_start_date + datetime.timedelta(days=random.choice(range(3))) # beginning of positioning
        date2 = date1 + (random.random() * (v.visit_end_date - date1))
        procedure_date = date1
        procedure_datetime = datetime.datetime.combine(procedure_date, datetime.datetime.min.time()) + datetime.timedelta(hours = random.choice(range(12)))
        procedure_concept_id = 4196006
        procedure_type_concept_id = 32817
        y = random.choice(range(1,x + 1))
        number = 1        
            
        for i in range(y + 1): # number of times of prone positioning
            
            list_of_procedures.append(Procedure_occurrence(person_id, procedure_concept_id,
                                                           procedure_type_concept_id, procedure_date,
                                                           procedure_datetime))
        
            prone_duration = random.choice(range(10,20))
            procedure_end_datetime = procedure_datetime + datetime.timedelta(hours = prone_duration)            
            procedure_end_date = procedure_end_datetime.date()
            
            back_duration = random.choice(range(4,16))
            procedure_datetime += datetime.timedelta(hours = prone_duration + back_duration)
            procedure_date = procedure_datetime.date()
            
            number += 1
            
            if number > y:
                break            
                
            #print("back_duration:", back_duration)
            
    return list_of_procedures

In [20]:
def insert(table, data):
    columns = ', '.join(data.keys())
    value_placeholder = ', '.join(['%s'] * len(data))
    sql = f"INSERT INTO {table} ({columns}) VALUES ({value_placeholder}) RETURNING {table}_id"
    cursor.execute(sql, list(data.values()))
    
    return cursor.fetchone()[0]

In [ ]:
# create patients and insert into DB
for person_id in patient_list:    
    ### CREATE
    
    # create person
    p = Person(person_id)
    print(p)
    
    #create visit
    v = Visit_occurrence(person_id)
    print(v)
    
    # create drugs
    list_of_drugs = create_drug_exp2(person_id, v, 10) # i set to 10
    print(list_of_drugs)
    
    # create first procedure
    prod = create_vent_params_procedure(person_id, v)
    print(prod)
    
    # create measurements
    list_of_measurements = create_vent_params_measurements(person_id, prod, v)    # REALLY USE THE FIRST PROD HERE??
    list_of_measurements += create_lab_values_measurements(person_id, v)
    print(list_of_measurements)
    
    # create rest of procdures 
    list_of_procedures = []
    if prod is not None:
        list_of_procedures.append(prod)
    
    list_of_procedures += create_pos_procedure(person_id, v, prod, x = 5)
    
    
    # break
    ### INSERT
    
    # insert person
    insert('person', asdict(p))
    con.commit()
    
    # insert visit
    insert('visit_occurrence', asdict(v))
    con.commit()
    
    # insert list of drugs
    for d in list_of_drugs:
        insert('drug_exposure', asdict(d))
        con.commit()
    
    # insert procedure
    for prod in list_of_procedures:
        insert('procedure_occurrence', asdict(prod))
        con.commit()
    
    # insert list of measurements
    for m in list_of_measurements:
        insert('measurement', asdict(m))
        con.commit()

    # NO condition_occurrence
    


Person(person_id=66, gender_concept_id=8521, year_of_birth=1972, month_of_birth=10, day_of_birth=21, race_concept_id=38003614, ethnicity_concept_id=0)
Visit_occurrence(person_id=66, visit_concept_id=32037, visit_start_date=datetime.date(2021, 1, 29), visit_end_date=datetime.date(2021, 2, 24), visit_type_concept_id=32817)
[Drug_exposure(person_id=66, drug_concept_id=19016072, drug_exposure_start_date=datetime.date(2021, 1, 31), drug_exposure_start_datetime=datetime.datetime(2021, 1, 31, 4, 0), drug_exposure_end_date=datetime.date(2021, 1, 31), drug_exposure_end_datetime=None, quantity=1000, drug_type_concept_id=32817), Drug_exposure(person_id=66, drug_concept_id=19016072, drug_exposure_start_date=datetime.date(2021, 1, 31), drug_exposure_start_datetime=datetime.datetime(2021, 1, 31, 9, 0), drug_exposure_end_date=datetime.date(2021, 1, 31), drug_exposure_end_datetime=None, quantity=3000, drug_type_concept_id=32817), Drug_exposure(person_id=66, drug_concept_id=19016072, drug_exposure_star

Person(person_id=67, gender_concept_id=8507, year_of_birth=1966, month_of_birth=8, day_of_birth=8, race_concept_id=38003598, ethnicity_concept_id=0)
Visit_occurrence(person_id=67, visit_concept_id=32037, visit_start_date=datetime.date(2020, 10, 23), visit_end_date=datetime.date(2020, 12, 13), visit_type_concept_id=32817)
[Drug_exposure(person_id=67, drug_concept_id=1315865, drug_exposure_start_date=datetime.date(2020, 10, 25), drug_exposure_start_datetime=datetime.datetime(2020, 10, 25, 14, 0), drug_exposure_end_date=datetime.date(2020, 10, 25), drug_exposure_end_datetime=None, quantity=2, drug_type_concept_id=32817), Drug_exposure(person_id=67, drug_concept_id=1315865, drug_exposure_start_date=datetime.date(2020, 10, 26), drug_exposure_start_datetime=datetime.datetime(2020, 10, 26, 0, 0), drug_exposure_end_date=datetime.date(2020, 10, 26), drug_exposure_end_datetime=None, quantity=3, drug_type_concept_id=32817), Drug_exposure(person_id=67, drug_concept_id=1315865, drug_exposure_start_

In [ ]:
con.close() # TODO: Not sure if works OR necessary ?